项目是文件系统中的一个文件夹。

Rope在需要时搜索项目中的Python模块和包。

重构只会更改项目中的文件和文件夹。

从项目内的模块导入的项目外模块会被处理，但不会被重构更改。

Rope在项目中创建了一个rope文件夹。默认情况下，此文件夹的名称为.ropeproject，但可以使用构造函数的ropefolder参数进行更改。传递“无”将阻止rope生成此文件夹。

Rope使用.ropeproject文件夹来保存对象信息和加载项目配置。

项目首选项可以通过向构造函数传递选项或在.ropeproject/config.py中进行配置。有关详细信息，请参阅默认的config.py，rope.base.default_config模块。

可以config.py文件中可用的所有配置指定为Project构造函数的关键字参数。这些参数将覆盖config.py文件中的参数。

每个项目都有一组被忽略的资源模式。你可以用它来告诉rope忽略匹配某些模式的文件和文件夹。

如果你不想丢失你的objectdb和历史记录，.ropeproject文件夹可以安全地复制到项目的其他克隆中。

libutils模块提供了一些工具，可以使rope作为一个库使用变得更加容易。稍后我们将详细讨论这个模块。

在rope中，项目中的文件和文件夹是通过rope.base.resources.Resource对象访问的。它有两个子类File和Folder。我们关心的是重构和修改（我们稍后会讨论它们）使用资源。

为项目中的路径创建资源有两个选项。第一种方法使用Project.get_resource（）方法。

```python
from rope.base import project
myresource = myproject.get_resource('/path/to/resource')
```

但是，最好使用libutils.path_to_resource（）函数，因为它更灵活，并且提供了一种创建资源的统一方法。它以项目(project)和路径(path)作为参数，并带有可选类型。type参数的值为file或folder，可以为尚不存在的对象创建资源。

```python
from rope.base import libutils
myresource = libutils.path_to_resource(myproject, '/path/to/resource')
```

当我们有一个资源。我们怎么去了解它呢？答案是使用它的path和real_path属性。资源。
Resource.real_path 是资源的绝对路径, Resource.path是项目的相对路径.

变量类型: 

PyFunction 函数

PyComprehension 推导式

PyClass 类

PyModule 模块

PyPackage 包

In [1]:
import rope
from rope.base.project import Project
from rope.refactor.rename import Rename
from rope.base import libutils
from rope.refactor.extract import ExtractVariable
from rope.contrib import codeassist
from rope.base.evaluate import eval_location

In [84]:
# 创建一个Rope项目
项目路径 = '/Users/yangxinyi/Downloads/200_临时文件夹/english_test_project_V2/'
项目 = Project(项目路径)
代码路径 = '/Users/yangxinyi/Downloads/200_临时文件夹/english_test_project_V2/tmp/test.py'

In [85]:
项目.find_module("channel")

<rope.base.resources.Folder "channel" at 0x1056b2550>

In [86]:
# 加载你想要重构的文件
# source = myproject.get_resource('config.py')

资源 = libutils.path_to_resource(项目, 代码路径)

下面展示如何从文件中提取变量, 首先需要指向某个文件的Resource对象: 

In [87]:
项目

<rope.base.project.Project "/Users/yangxinyi/Downloads/200_临时文件夹/english_test_project_V2">

In [88]:
资源.read()

'import json\nimport pandas as pd\n\nfrom math import *\nimport math\n\nfrom bridge import bridge\n\n# A = 1\nclass A:\n    def __init__(self, B):\n        self.C = B\n\n    def t(self, K):\n        O = K + 1\n        return O\n\n\ndef f(x):\n    y = x + 1\n    return y\n\n\nk = pow(10)\nprint(k)\n\nd = pd.array(3)\n\n\nb = bridge.Bridge()\n\n\n\n\n\n\n'

In [89]:
# 获取 Rope 资源
# resource = myproject.get_file('config.py')

In [107]:
# 寻找特定函数名的位置
offset = 资源.read().index('bridge')
offset

70

In [108]:
监控范围 = libutils.get_string_scope(项目, 资源.read(), 资源)

In [109]:
监控范围.get_names()  # 当前监控范围变量

{'ArithmeticError': <rope.base.builtins.BuiltinName at 0x104b81650>,
 'AssertionError': <rope.base.builtins.BuiltinName at 0x105098490>,
 'AttributeError': <rope.base.builtins.BuiltinName at 0x105098590>,
 'BaseException': <rope.base.builtins.BuiltinName at 0x1050985d0>,
 'BaseExceptionGroup': <rope.base.builtins.BuiltinName at 0x105098650>,
 'BlockingIOError': <rope.base.builtins.BuiltinName at 0x1050986d0>,
 'BrokenPipeError': <rope.base.builtins.BuiltinName at 0x105098750>,
 'BufferError': <rope.base.builtins.BuiltinName at 0x1050987d0>,
 'BytesWarning': <rope.base.builtins.BuiltinName at 0x105098850>,
 'ChildProcessError': <rope.base.builtins.BuiltinName at 0x1050988d0>,
 'ConnectionAbortedError': <rope.base.builtins.BuiltinName at 0x105098950>,
 'ConnectionError': <rope.base.builtins.BuiltinName at 0x1050989d0>,
 'ConnectionRefusedError': <rope.base.builtins.BuiltinName at 0x105098a50>,
 'ConnectionResetError': <rope.base.builtins.BuiltinName at 0x105098ad0>,
 'DeprecationWarning'

In [110]:
监控范围.get_defined_names()  # 当前监控范围变量

{'json': <rope.base.pynamesdef.ImportedModule at 0x1057a5fd0>,
 'pd': <rope.base.pynamesdef.ImportedModule at 0x1057c26d0>,
 'math': <rope.base.pynamesdef.ImportedModule at 0x1057c1f50>,
 'bridge': <rope.base.pynamesdef.ImportedName at 0x1057ac890>,
 'A': <rope.base.pynamesdef.DefinedName at 0x105571a90>,
 'f': <rope.base.pynamesdef.DefinedName at 0x1057ad450>,
 'k': <rope.base.pynamesdef.AssignedName at 0x10579a450>,
 'd': <rope.base.pynamesdef.AssignedName at 0x1057acfd0>,
 'b': <rope.base.pynamesdef.AssignedName at 0x1057ae9d0>}

In [111]:
dir(监控范围)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_calculate_scope_regions',
 '_calculate_scope_regions_for_module',
 '_create_scopes',
 '_get_global_scope',
 '_get_source',
 '_propagated_lookup',
 '_scope_finder',
 'builtin_names',
 'end',
 'get_body_start',
 'get_defined_names',
 'get_end',
 'get_inner_scope_for_line',
 'get_inner_scope_for_offset',
 'get_kind',
 'get_logical_end',
 'get_name',
 'get_names',
 'get_propagated_names',
 'get_region',
 'get_scopes',
 'get_start',
 'in_region',
 'logical_end',
 'lookup',
 'names',
 'parent',
 'pycore',
 'pyobject',
 'start']

In [112]:
模块 = libutils.get_string_module(项目, 资源.read(), 资源)

In [113]:
py_变量 = eval_location(模块, offset)
py_变量

In [114]:
dir(py_变量)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_current_folder',
 '_get_pymodule',
 'get_definition_location',
 'get_object',
 'importing_module',
 'level',
 'module_name',
 'pymodule',
 'resource']

In [115]:
变量对象 = py_变量.get_object()
变量对象

<rope.base.pyobjectsdef.PyPackage "bridge" at 0x10566a490>

In [116]:
py_变量.get_definition_location()

(<rope.base.pyobjectsdef.PyPackage "bridge" at 0x10566a490>, 1)

In [106]:
dir(变量对象)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_create_concluded_attributes',
 '_create_scope',
 '_create_structural_attributes',
 '_get_base_type',
 '_get_bases',
 '_get_concluded_attributes',
 '_get_defined_objects',
 '_get_structural_attributes',
 '_superclasses',
 '_types',
 '_unknown',
 'absolute_name',
 'ast_node',
 'attributes',
 'concluded_attributes',
 'defineds',
 'get_ast',
 'get_attribute',
 'get_attributes',
 'get_doc',
 'get_module',
 'get_name',
 'get_scope',
 'get_superclasses',
 'get_type',
 'parent',
 'pycore',
 'scope',
 'structural_attributes',
 'type',
 'visitor_class']

In [102]:
变量对象.get_type()

In [103]:
t = a.get_type()

NameError: name 'a' is not defined

In [31]:
type(a)

rope.base.pyobjectsdef.PyClass

In [32]:
type(t)

rope.base.pyobjects.PyObject

In [127]:
a.get_name

'Config'

In [134]:
result = codeassist.get_definition_location(myproject, content, offset, resource)
result

(None, None)

In [ ]:
result

In [129]:
file

<rope.base.resources.File "config.py" at 0x125dcaa10>

In [133]:
result = codeassist.get_doc(myproject, content, offset, resource)
print(result)

None


In [35]:
extractor = ExtractVariable(myproject, resource, offset, end)

In [36]:
extractor

然后，我们可以计算变化：

changes = extractor.get_changes('extracted_variable')每次重构都返回一个rope.base.change.Change对象，该对象保存了它所做的更改。计算这些变化可能很耗时

In [38]:
changes = extractor.get_changes('extracted_variable')

如上一节所述，每次重构都会返回一个rope.base.change.Change对象。现在我们如何知道它包含什么以及如何执行它？

你可以使用changes.get_description（）来获得预览。当你不太关心格式时，它很有用。否则，您可以直接使用更改对象。请参阅rope.base.change模块中的文档。

In [40]:
print(changes.get_description())

Extract variable <extracted_variable>:


--- a/config.py
+++ b/config.py
@@ -9,7 +9,8 @@
 
 # 将所有可用的配置项写在字典里, 请使用小写字母
 # 此处的配置值无实际意义，程序不会读取此处的配置，仅用于提示格式，请将配置加入到config.json中
-new_variable_name = {
+extracted_variable = new_variable_name
+extracted_variable = {
     # openai api配置
     "open_ai_api_key": "",  # openai api key
     # openai apibase，当use_azure_chatgpt为true时，需要设置对应的api base




执行:

myproject.do(changes)

第一种方法是使用rope.base.fscommands（请参阅编写文件系统命令）。可以像以前一样使用Project.do（）执行更改。

第二种方法是根据返回的changes对象手动执行更改（再次参见rope.base.change模块中的文档）。如果使用这种方法，则无法使用project.history.undo（）撤销重构。

通常编辑器需要重新加载通过rope更改的文件。您可以使用 Change.get_changed_resources() 来获取需要重新加载的资源列表。

In [41]:
myproject.do(changes)

In [42]:
myproject.validate(resource)

In [43]:
myproject.close()

In [47]:
pymodule = libutils.get_string_module(myproject, resource)

TypeError: argument of type 'File' is not iterable

In [45]:
libutils.analyze_modules(myproject)

In [9]:
filename = Project_path + "/" + "config.py"
with open(filename, 'r') as file:
    content = file.read()

# 寻找特定函数名的位置
offset = content.index('available_setting')
offset

181

In [10]:
# 创建一个重命名重构
rename = Rename(project, module, offset)
# offset 是变量名在文件中的位置

In [11]:
# 执行重命名操作
change = rename.get_changes(new_name='new_variable_name')
project.do(change)

In [12]:
import rope.base.project
import rope.refactor.rename

In [13]:
Project_path = '/Users/yangxinyi/Downloads/200_临时文件夹/english_test_project_V2'
project = Project(Project_path)

In [14]:
# 指定要重命名的变量的文件和位置
file_path = 'app.py'

In [19]:
filename = Project_path + "/" + "app.py"
with open(filename, 'r') as file:
    content = file.read()

# 寻找特定函数名的位置
offset = content.index('os')
offset

25

In [ ]:
# 加载文件
resource = project.get_file(file_path)

In [ ]:
# 创建重命名重构对象
renamer = rope.refactor.rename.Rename(project, resource, offset)

In [22]:
# 获取重命名变更
new_name = '系统'
change = renamer.get_changes(new_name)

# 执行重构
project.do(change)

# 保存项目，确保所有更改被写入文件
project.close()


## Restructuring 重构案例

In [48]:
from rope.base.project import Project
project = Project('.')

In [49]:
mod1 = project.root.create_file('mod1.py')

In [50]:
mod1.write('def pow(x, y):\n    result = 1\n'
           '    for i in range(y):\n        result *= x\n'
           '    return result\n')
mod2 = project.root.create_file('mod2.py')
mod2.write('import mod1\nprint(mod1.pow(2, 3))\n')

In [51]:
from rope.refactor import restructure

In [52]:
pattern = '${pow_func}(${param1}, ${param2})'

In [53]:
goal = '${param1} ** ${param2}'

In [54]:
args = {'pow_func': 'name=mod1.pow'}

In [55]:
restructuring = restructure.Restructure(project, pattern, goal, args)

In [56]:
project.do(restructuring.get_changes())

In [57]:
mod1.remove()

error: the following file has staged content different from both the
file and the HEAD:
    mod1.py
(use -f to force removal)


## autoimport 查找模块

In [92]:
from rope.base.project import Project
from rope.contrib.autoimport.sqlite import AutoImport

project = Project("/Users/yangxinyi/Downloads/200_临时文件夹/english_test_project_V2")
autoimport = AutoImport(project, memory=False)


In [94]:
autoimport.generate_cache()

In [95]:
autoimport.generate_modules_cache()

Missing parentheses in call to 'print'. Did you mean print(...)? (<string>, line 116)
leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<string>, line 31)
invalid syntax (<string>, line 27)
multiple exception types must be parenthesized (<string>, line 93)
multiple exception types must be parenthesized (<string>, line 63)
multiple exception types must be parenthesized (<string>, line 126)
multiple exception types must be parenthesized (<string>, line 59)
Missing parentheses in call to 'print'. Did you mean print(...)? (<string>, line 157)
Missing parentheses in call to 'print'. Did you mean print(...)? (<string>, line 3)
Missing parentheses in call to 'print'. Did you mean print(...)? (<string>, line 40)
invalid syntax (<string>, line 136)
Missing parentheses in call to 'print'. Did you mean print(...)? (<string>, line 2)
invalid syntax (<string>, line 32)
Missing parentheses in call to 'print'. Did you mean print(...)? (<string>, line 20

In [100]:
list(autoimport.search_full("os"))

[SearchResult(import_statement='from pydevd_attach_to_process.winappdbg.win32.shlwapi import OS_DATACENTER', name='OS_DATACENTER', source=5, itemkind=6),
 SearchResult(import_statement='from pydevd_attach_to_process.winappdbg.win32.shlwapi import OS_HOME', name='OS_HOME', source=5, itemkind=6),
 SearchResult(import_statement='from pydevd_attach_to_process.winappdbg.win32.shlwapi import OS_PROFESSIONAL', name='OS_PROFESSIONAL', source=5, itemkind=6),
 SearchResult(import_statement='from PyObjCTools.TestSupport import os_level_between', name='os_level_between', source=4, itemkind=3),
 SearchResult(import_statement='from pydevd_attach_to_process.winappdbg.win32.version import OS_UNKNOWN', name='OS_UNKNOWN', source=5, itemkind=6),
 SearchResult(import_statement='from pydevd_attach_to_process.winappdbg.win32.shlwapi import OS_SERVERADMINUI', name='OS_SERVERADMINUI', source=5, itemkind=6),
 SearchResult(import_statement='from pydevd_attach_to_process.winappdbg.win32.shlwapi import OS_PERSONA

In [98]:
print(autoimport.search("os"))

[('from lib.itchat.config import OS', 'OS'), ('from lib2to3.tests.data.infinite_recursion import OSBigEndian', 'OSBigEndian'), ('from lib2to3.tests.data.infinite_recursion import OSUnknownByteOrder', 'OSUnknownByteOrder'), ('from tty import OSPEED', 'OSPEED'), ('import os', 'os'), ('from lib2to3.tests.data.infinite_recursion import OSLittleEndian', 'OSLittleEndian'), ('from PyObjCTools.TestSupport import os_level_between', 'os_level_between'), ('from setuptools.tests.compat.py39 import os_helper', 'os_helper'), ('from dashscope.utils.oss_utils import OssUtils', 'OssUtils'), ('from PyObjCTools.TestSupport import os_level_key', 'os_level_key'), ('from importlib_resources.tests.compat.py39 import os_helper', 'os_helper'), ('from IPython.lib.clipboard import osx_clipboard_get', 'osx_clipboard_get'), ('from qtconsole.ansi_code_processor import OSC_SUBPATTERN', 'OSC_SUBPATTERN'), ('from pip._vendor.distro import os_release_attr', 'os_release_attr'), ('from numpy.compat.py3k import os_PathLik

In [85]:
autoimport.get_all_names()

{'ADMIN_COMMANDS',
 'ASYNC_COMPONENTS',
 'ATTACHMENT',
 'AbstractUserDict',
 'AliQwenBot',
 'AliQwenSession',
 'AliVoice',
 'AliyunTokenGenerator',
 'AttributeDict',
 'AzureChatGPTBot',
 'AzureVoice',
 'BAIDU',
 'BAIDU_API_KEY',
 'BAIDU_SECRET_KEY',
 'BASE_URL',
 'BDunit',
 'BLOCK',
 'BaiduTranslator',
 'BaiduUnitBot',
 'BaiduVoice',
 'BaiduWenxinBot',
 'BaiduWenxinSession',
 'Banwords',
 'Bot',
 'Bridge',
 'CARD',
 'CHATGPT',
 'CHATGPTONAZURE',
 'CLAUDEAI',
 'COMMANDS',
 'Channel',
 'ChatChannel',
 'ChatClient',
 'ChatGPTBot',
 'ChatGPTSession',
 'ChatMessage',
 'Chatroom',
 'ChatroomMember',
 'ClaudeAIBot',
 'ClaudeAiSession',
 'Config',
 'ContactList',
 'Context',
 'ContextType',
 'Core',
 'DEFAULT_QR',
 'DINGTALK',
 'DIR',
 'Dequeue',
 'DingTalkChanel',
 'DingTalkMessage',
 'Dungeon',
 'ElevenLabsVoice',
 'Event',
 'EventAction',
 'EventContext',
 'ExpiredDict',
 'FEISHU',
 'FIRST_MSG',
 'FRIENDS',
 'FeiShuChanel',
 'FeishuController',
 'FeishuMessage',
 'Finish',
 'GEMINI',
 'GPT3

In [90]:
autoimport.get_name_locations("CHATGPT")

[]

In [76]:
autoimport.names

{'factory': ['create_translator'],
 'itchat.async_components.login': ['logger',
  'load_login',
  'login',
  'push_login',
  'get_QRuuid',
  'get_QR',
  'check_login',
  'process_login_info',
  'web_init',
  'show_mobile_login',
  'start_receiving',
  'sync_check',
  'get_msg',
  'logout'],
 'ali_voice': ['AliVoice'],
 'memory': ['USER_IMAGE_CACHE'],
 'plugins.finish.finish': ['Finish'],
 'plugins.tool.tool': ['Tool'],
 'itchat.components.messages': ['logger',
  'load_messages',
  'get_download_fn',
  'produce_msg',
  'produce_group_chat',
  'send_raw_msg',
  'send_msg',
  'upload_file',
  'upload_chunk_file',
  'send_file',
  'send_image',
  'send_video',
  'send',
  'revoke'],
 'pytts_voice': ['PyttsVoice'],
 'bot.bot': ['Bot'],
 'novel_send_script_cobox': ['current_file_path',
  'current_dir',
  'NovelHandle',
  'QQEmailSend',
  'random_sleep',
  'random_sleep_seconds',
  'novel_handle',
  'f',
  'chapter_has_read_num',
  'save_merge_file_name',
  'content',
  'qq_email_send',
  're

## 类型分析

In [37]:
py_module = libutils.get_string_module(myproject, resource.read(), resource)

In [38]:
resource

<rope.base.resources.File "tmp/test.py" at 0x1062ca410>

In [39]:
defined_names = py_module.get_scope().get_names()

In [40]:
defined_names

{'ArithmeticError': <rope.base.builtins.BuiltinName at 0x105b16ed0>,
 'AssertionError': <rope.base.builtins.BuiltinName at 0x105b19d10>,
 'AttributeError': <rope.base.builtins.BuiltinName at 0x106240dd0>,
 'BaseException': <rope.base.builtins.BuiltinName at 0x106240f50>,
 'BaseExceptionGroup': <rope.base.builtins.BuiltinName at 0x106241590>,
 'BlockingIOError': <rope.base.builtins.BuiltinName at 0x106241390>,
 'BrokenPipeError': <rope.base.builtins.BuiltinName at 0x1062415d0>,
 'BufferError': <rope.base.builtins.BuiltinName at 0x106241610>,
 'BytesWarning': <rope.base.builtins.BuiltinName at 0x106241810>,
 'ChildProcessError': <rope.base.builtins.BuiltinName at 0x106241890>,
 'ConnectionAbortedError': <rope.base.builtins.BuiltinName at 0x106241950>,
 'ConnectionError': <rope.base.builtins.BuiltinName at 0x106241990>,
 'ConnectionRefusedError': <rope.base.builtins.BuiltinName at 0x1062419d0>,
 'ConnectionResetError': <rope.base.builtins.BuiltinName at 0x106241d50>,
 'DeprecationWarning'

In [42]:
d = defined_names["f"]

In [44]:
top_scope = py_module.get_scope()

In [51]:
top_scope.get_name("f")

In [55]:
f = top_scope.get_inner_scope_for_line(15)

In [58]:
f

In [57]:
f.get_names()

{'y': <rope.base.pynamesdef.AssignedName at 0x1063ae3d0>,
 'x': <rope.base.pynamesdef.ParameterName at 0x1062877d0>}

{'ArithmeticError': <rope.base.builtins.BuiltinName at 0x105b16ed0>,
 'AssertionError': <rope.base.builtins.BuiltinName at 0x105b19d10>,
 'AttributeError': <rope.base.builtins.BuiltinName at 0x106240dd0>,
 'BaseException': <rope.base.builtins.BuiltinName at 0x106240f50>,
 'BaseExceptionGroup': <rope.base.builtins.BuiltinName at 0x106241590>,
 'BlockingIOError': <rope.base.builtins.BuiltinName at 0x106241390>,
 'BrokenPipeError': <rope.base.builtins.BuiltinName at 0x1062415d0>,
 'BufferError': <rope.base.builtins.BuiltinName at 0x106241610>,
 'BytesWarning': <rope.base.builtins.BuiltinName at 0x106241810>,
 'ChildProcessError': <rope.base.builtins.BuiltinName at 0x106241890>,
 'ConnectionAbortedError': <rope.base.builtins.BuiltinName at 0x106241950>,
 'ConnectionError': <rope.base.builtins.BuiltinName at 0x106241990>,
 'ConnectionRefusedError': <rope.base.builtins.BuiltinName at 0x1062419d0>,
 'ConnectionResetError': <rope.base.builtins.BuiltinName at 0x106241d50>,
 'DeprecationWarning'

In [89]:
def print_scope_details(scope, indent=0):
    indent_space = ' ' * indent
    print(f"{indent_space}Scope: {scope.get_kind()} (Start: {scope.get_start()}, End: {scope.get_end()})")
    print(f"{indent_space}Variables:")
    for name, pyname in scope.get_names().items():
        print(f"{indent_space}  {name}: {type(pyname).__name__}")

    # 递归打印子作用域的信息
    subscopes = scope.get_scopes()
    for subscope in subscopes:
        print_scope_details(subscope, indent + 2)

In [90]:
print_scope_details(top_scope)

Scope: Module (Start: 1, End: 17)
Variables:
  ArithmeticError: BuiltinName
  AssertionError: BuiltinName
  AttributeError: BuiltinName
  BaseException: BuiltinName
  BaseExceptionGroup: BuiltinName
  BlockingIOError: BuiltinName
  BrokenPipeError: BuiltinName
  BufferError: BuiltinName
  ChildProcessError: BuiltinName
  ConnectionAbortedError: BuiltinName
  ConnectionError: BuiltinName
  ConnectionRefusedError: BuiltinName
  ConnectionResetError: BuiltinName
  EOFError: BuiltinName
  Ellipsis: BuiltinName
  EnvironmentError: BuiltinName
  Exception: BuiltinName
  ExceptionGroup: BuiltinName
  False: BuiltinName
  FileExistsError: BuiltinName
  FileNotFoundError: BuiltinName
  FloatingPointError: BuiltinName
  GeneratorExit: BuiltinName
  IOError: BuiltinName
  ImportError: BuiltinName
  IndentationError: BuiltinName
  IndexError: BuiltinName
  InterruptedError: BuiltinName
  IsADirectoryError: BuiltinName
  KeyError: BuiltinName
  KeyboardInterrupt: BuiltinName
  LookupError: BuiltinN

In [16]:
# 过滤出自定义变量（排除内置名称）
custom_vars = []
for name in defined_names:
    if defined_names[name].is_defined() and not defined_names[name].get_object().is_builtin():  # 检查是否为自定义定义且不是内置的
        py_object = name.get_object()
        if py_object.get_kind() == 'variable':
            custom_vars.append(name)

AttributeError: 'BuiltinName' object has no attribute 'is_defined'

In [33]:
b = defined_names["pandas"]

KeyError: 'pandas'

In [25]:
b.get_definition_location()

(<rope.base.pyobjectsdef.PyModule "test" at 0x105b15ed0>, 3)

In [18]:
type(b)

rope.base.pynamesdef.AssignedName

In [20]:
c = b.get_object()

In [22]:
c.get_type()

In [59]:
import rope
from rope.base.project import Project
from rope.base import ast, pyobjects, evaluate

In [68]:
myproject.get_resource("tmp/test.py").get_python_object()

AttributeError: 'File' object has no attribute 'get_python_object'

In [61]:
pyobject = myproject.get_pyobject(resource)

AttributeError: 'Project' object has no attribute 'get_pyobject'

In [63]:
myproject

<rope.base.project.Project "/Users/yangxinyi/Downloads/200_临时文件夹/english_test_project_V2">

In [77]:
tree = ast.parse(resource.read())

In [78]:
ast.walk(ast.parse(resource.read()))

<generator object walk at 0x1067304f0>

In [81]:
for i in ast.walk(ast.parse(resource.read())):
    print(i)

In [75]:
a = list(ast.walk(ast.parse(resource.read())))

In [82]:
b = a[0]

In [84]:
b

AttributeError: 'Module' object has no attribute 'id'

In [92]:
libutils.analyze_module(myproject, resource)

In [93]:
libutils.modname(resource)

'test'

In [96]:
import importlib.util
import pkgutil
import sys

In [97]:
sys.builtin_module_names

('_abc',
 '_ast',
 '_codecs',
 '_collections',
 '_functools',
 '_imp',
 '_io',
 '_locale',
 '_operator',
 '_signal',
 '_sre',
 '_stat',
 '_string',
 '_symtable',
 '_thread',
 '_tokenize',
 '_tracemalloc',
 '_warnings',
 '_weakref',
 'atexit',
 'builtins',
 'errno',
 'faulthandler',
 'gc',
 'itertools',
 'marshal',
 'posix',
 'pwd',
 'sys',
 'time',
 'xxsubtype')

In [101]:
importlib.util.find_spec("re").origin

'/Users/yangxinyi/opt/miniforge3/envs/code_chinese_to_english/lib/python3.11/re/__init__.py'

In [103]:
module_spec = importlib.util.find_spec("channel")
module_spec

In [104]:
module_path = module_spec.origin
module_path

AttributeError: 'NoneType' object has no attribute 'origin'

In [105]:
import ast

code = """
import json
import pandas as pd
from os import path as os_path
"""

class ImportExtractor(ast.NodeVisitor):
    def __init__(self):
        self.imports = []

    def visit_Import(self, node):
        for alias in node.names:
            self.imports.append((alias.name, alias.asname if alias.asname else alias.name))
        self.generic_visit(node)

    def visit_ImportFrom(self, node):
        for alias in node.names:
            module_name = f"{node.module}.{alias.name}" if node.module else alias.name
            self.imports.append((module_name, alias.asname if alias.asname else alias.name))
        self.generic_visit(node)

# 解析代码
tree = ast.parse(code)
extractor = ImportExtractor()
extractor.visit(tree)

# 输出结果
print(extractor.imports)


[('json', 'json'), ('pandas', 'pd'), ('os.path', 'os_path')]
